# 4.7 生成文本
在本章的最后一节中，我们将实现将 GPT 模型的张量输出转换回文本的代码。在开始之前，让我们简要回顾一下像 LLM 这样的生成模型是如何一次生成一个单词（或标记）的，如图 4.16 所示。

**图 4.16 此图解说明了 LLM 一次生成一个标记的逐步过程。从初始输入上下文（“Hello, I am”）开始，模型在每次迭代中预测一个后续标记，并将其添加到下一轮预测的输入上下文中。如图所示，第一次迭代添加了“a”，第二次“model”，第三次“ready”，逐步构建句子。**

![fig4.16](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-4-16.jpg?raw=true)

图 4.16 说明了 GPT 模型根据输入上下文（例如 “Hello, I am”）生成文本的逐步过程，从宏观层面上看。随着每次迭代，输入上下文增长，使模型能够生成连贯且上下文恰当的文本。到第六次迭代时，模型已经构建了一个完整的句子：“Hello, I am a model ready to help.”
在前一节中，我们看到我们当前的 GPTModel 实现输出的张量形状为 [batch_size, num_token, vocab_size]。现在，问题是，GPT 模型如何从这些输出张量变为图 4.16 所示的生成文本？
GPT 模型从输出张量到生成文本的过程涉及几个步骤，如图 4.17 所示。这些步骤包括解码输出张量，根据概率分布选择标记，并将这些标记转换为人类可读的文本。

**图 4.17 详细描述了 GPT 模型中文本生成的机制，展示了标记生成过程中的单次迭代。过程从将输入文本编码为标记 ID 开始，然后将其输入到 GPT 模型中。然后将模型的输出转换回文本并附加到原始输入文本中。**
![fig4.17](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-4-17.jpg?raw=true)

图 4.17 中详细的下一个标记生成过程说明了 GPT 模型给定输入后生成下一个标记的单个步骤。

在每个步骤中，模型输出一个表示潜在下一个标记的向量矩阵。提取对应于下一个标记的向量，并通过 softmax 函数将其转换为概率分布。在包含结果概率分数的向量中，找到最高值的索引，该索引转换为标记 ID。然后将此标记 ID 解码回文本，生成序列中的下一个标记。最后，将此标记附加到先前的输入中，形成下一次迭代的新输入序列。这个逐步过程使得模型能够顺序地生成文本，从初始输入上下文构建连贯的短语和句子。

实际上，我们重复此过程多次迭代，如图 4.16 早先所示，直到我们达到用户指定的生成标记数量。

在代码中，我们可以如下实现标记生成过程：

**清单 4.8 GPT 模型生成文本的函数**

```
def generate_text_simple(model, idx, max_new_tokens, context_size): #A
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:] #B
        with torch.no_grad():
            logits = model(idx_cond)
            
        logits = logits[:, -1, :] #C
        probas = torch.softmax(logits, dim=-1) #D
        idx_next = torch.argmax(probas, dim=-1, keepdim=True) #E
        idx = torch.cat((idx, idx_next), dim=1) #F
    return idx

提供的代码片段展示了使用 PyTorch 的语言模型的一个简单生成循环的实现。它迭代生成指定数量的新标记，裁剪当前上下文以适应模型的最大上下文大小，计算预测然后选择基于最高概率预测的下一个标记。

在前面的代码中，generate_text_simple 函数使用 softmax 函数将 logits 转换为概率分布，从中我们通过 torch.argmax 确定最高值的位置。softmax 函数是单调的，意味着它在转换为输出时保留其输入的顺序。因此，实际上，softmax 步骤是多余的，因为 softmax 输出张量中得分最高的位置与 logits 张量中的位置相同。换句话说，我们可以直接将 torch.argmax 函数应用于 logits 张量，并获得相同的结果。

然而，我们编写了转换来说明将 logits 转换为概率的完整过程，这可以增加额外的直觉，例如模型生成最有可能的下一个标记，这被称为贪婪解码。

在下一章中，当我们实现 GPT 训练代码时，我们还将引入额外的采样技术，我们修改 softmax 输出，使得模型不总是选择最有可能的标记，这在生成文本中引入了变异性和创造性。

这个使用 generate_text_simple 函数一次生成一个标记 ID 并将其附加到上下文的过程在图 4.18 中进一步说明。（每次迭代的标记 ID 生成过程在图 4.17 中详细说明。）

**图 4.18 描述六次标记预测周期的迭代过程，其中模型接受初始标记 ID 序列作为输入，预测下一个标记，并将此标记附加到下一次迭代的输入序列中。（标记 ID 也翻译为相应的文本以便更好地理解。）**

![fig4.18](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-4-18.jpg?raw=true)

如图 4.18 所示，我们以迭代方式生成标记 ID。

例如，在第一次迭代中，模型被提供与“Hello , I am”对应的标记，预测下一个标记（ID 为 257，即“a”），并将其附加到输入中。这个过程重复，直到模型在六次迭代后生成完整的句子“Hello, I am a model ready to help.”

现在让我们实际使用图 4.18 中所示的“Hello, I am”上下文作为模型输入来尝试 generate_text_simple 函数。

首先，我们将输入上下文编码为标记 ID：


```
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0) #A
print("encoded_tensor.shape:", encoded_tensor.shape)

编码的ID如下:

```
encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])

接下来，我们将模型置于 `.eval()` 模式，这将禁用像 dropout 这样的随机组件，这些组件仅在训练期间使用，并在编码的输入张量上使用 `generate_text_simple` 函数：

````
model.eval() #A
out = generate_text_simple(
model=model,
idx=encoded_tensor,
max_new_tokens=6,
context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))

生成的输出标记 ID 如下：

````
Output: tensor([[15496, 11, 314, 716, 27018, 24086, 47843, 30961, 42348, 7267]])
Output length: 10

使用分词器的 `.decode` 方法，我们可以将这些 ID 转换回文本：

````
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

模型输出的文本格式如下：

````
Hello, I am Featureiman Byeswickattribute argue logger Normandy Co


正如我们所见，根据前面的输出，模型生成了无意义的文字，这与图 4.18 中显示的连贯文本完全不同。发生了什么？模型无法生成连贯文本的原因是我们还没有对其进行训练。到目前为止，我们只是实现了 GPT 架构并用初始随机权重初始化了一个 GPT 模型实例。

模型训练本身是一个大话题，我们将在下一章中处理它。


**练习 4.3：使用独立的 dropout 参数**

在本章的开始，我们在 GPT_CONFIG_124M 字典中定义了一个全局的 "drop_rate" 设置，以设置 GPTModel 架构中各处的 dropout 率。修改代码以为模型架构中各个 dropout 层指定不同的 dropout 值。（提示：有三个不同的地方使用了 dropout 层：嵌入层、快捷连接层和多头注意力模块。）
